# Scraping Job Data from Indeed.com

First we will need to import several libraries

In [117]:
from re import A
import matplotlib as plt
import pandas as pd
from urllib.request import urlopen
import urllib
from bs4 import BeautifulSoup
from urllib.parse import quote
import re
import pandas as pd

## Define a class that will store job posts on Indeed
All the relevant data

In [118]:
# this is just a utility function for printing to the console
def printhr():
    print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")

# This class will be created from data scraped from python
# and converted to csv
class JobPost:
    def __init__(self, job_id, job_title, job_company, job_salary,
                 location, job_date_posted, job_company_link, job_page_title):
        self.job_id = job_id
        self.job_title = job_title
        self.job_company = job_company
        self.job_date_posted = job_date_posted
        self.job_company_link = job_company_link
        self.job_salary = job_salary
        self.location = location
        self.job_page_title = job_page_title

    # prints to console for debugging
    def printF(self):
        print(self.job_title)
        print(f"{self.job_company} at {self.location}")
        print(self.job_date_posted)
        print(self.job_salary)
        print(self.job_page_title)
        print(self.job_company_link)
        printhr()

    # for debugging with less text
    def desc(self):
        print(self.job_title)
        print(f"{self.job_company} at {self.location}")

    def to_dict(self):
        return {
            'job_id': self.job_id,
            'job_title': self.job_title,
            'job_company': self.job_company,
            'job_date_posted': self.job_date_posted,
            'job_company_link': self.job_company_link,
            'job_salary': self.job_salary,
            'location': self.location,
            'job_page_title': self.job_page_title
        }

# Instantiate a list
This will contain all the job posts that we collect during the scrape

In [119]:
jobs_post_list = []

## Specify some keywords that we will search for

In [120]:
keywords = ["bootcamp", "SQL", "Tableau", "degree", "data"]

# Ask user for specific inputs
This section requests specific inputs from the user and may not be necessary. running on default values for now

In [121]:
#use_default_vals = input("Use default config? (Y/N)")
use_default_vals = "Y"
while True:
  if (use_default_vals.upper() == "Y"):
    print("using default vals")
    job_query = quote("data-analyst")
    CompFilter = True
    n = 15
    deg = quote("no degree")
    url = "https://www.indeed.com/jobs?q=" + \
    job_query + "&" + deg
    from_fs = False
    print(f'Searching url {url}')
    break
  elif (use_default_vals.upper() == "N"):
    print("input your own options to the parser")
    break
  else:
    "did not recognize that. Your options are Y/N"

using default vals
Searching url https://www.indeed.com/jobs?q=data-analyst&no%20degree


In [122]:
if from_fs:
  html = open("pass1.html", "r", encoding="utf8")
else: 
  html = urllib.request.urlopen(url).read()
soup = BeautifulSoup(html, "html.parser")

job_lis = soup.find("ul", "jobsearch-ResultsList")

# SEPARATE VALUES RETURNED INTO DIVS THAT CONTAIN BOTH
# .jobCard_mainContent AND .jobCardShelfContainer
result_card_list = soup.find_all("result")

# FIND THE PARENT TAGS BY CSS CLASS
results_list = soup.find_all("div", class_="cardOutline")
#print(f"Found {len(results_list)} job search results items \n")

## Iterate Through the <table> Tag that contains job

In [123]:
# ITERATE THROUGH THE <table> TAGS THAT CONTAIN JOB DATA
i = 0

print(f'Found {len(results_list)} results')
for result in results_list:
    # the for loop will be automatically be limited by the pagination on indeed
    # limited to 15 results
    if i < n:  # iterate for as many times as was specified by the config

        # EXTRACT THE SALARY IF IT IS SHOWN ON THE JOBCARD
        salary_divs = result.find("div", "salary-snippet-container")
        salary = "Compensation Not Listed"
        if salary_divs is not None:
            salary_sub_div = salary_divs.find("div")
            for s in salary_sub_div.strings:
                if s[0] == '$':
                    salary = s

        # EXTRACT THE JOB TITLE
        title_h2 = result.find_all("h2", class_="jobTitle")
        a_tag = title_h2[0].find("a")
        title = a_tag.string

        # EXTRACT JOB ID FROM DATA ATTRIBUTE
        job_id = a_tag["data-jk"]

        # EXTRACT THE COMPANY
        company_span = result.find_all("span", class_="companyName")
        for span in company_span:
            if span.string[0] != '+':
                company = span.string
            else:
                company = span[1].string

        # EXTRACT THE URL FOR THE JOB APPLICATION
        company_link = result.find("a", class_="jcs-JobTitle")['href']
        company_link = 'http://indeed.com' + company_link

        # EXTRACT THE LOCATION
        location_div = result.find("div", "companyLocation")
        location = location_div.string
        if location is None:
            location = "not specified"

        # EXTRACT THE TIME THE JOB WAS POSTED date_posted =
        date_posted = "Not specified when posted"
        date_spans = result.find_all("span", class_="date")
        for d_span in date_spans:
            if (d_span.string is not None):
                date_posted = d_span.string
        try:
            # Create a url for the job page using indeed's url query string
            job_page_url = "https://www.indeed.com/viewjob?jk=" + job_id
            #print(f'searching at job url {job_page_url}')

            # Create a new soup of job_page_url
            job_html = urllib.request.urlopen(job_page_url).read()
            job_soup = BeautifulSoup(job_html, "html.parser")
            # Search the html soup for mentions of a specific keyword
            for keyword in keywords:
                data_mentions = job_soup.body.findAll(text=re.compile(keyword))
                print(f'{title} contains {len(data_mentions)} mentions of {keyword}')
            print(job_page_url)

            # IF THE SINGLE PAGE VIEW OF THE JOB, EXTRACT MORE DATA
            posted = job_soup.find(
                "span", class_="jobsearch-HiringInsights-entry--text").string

            # Instantiate JobPost class and append it to list
            job = JobPost(job_id, title, company, salary, location,
                          date_posted, company_link, job_soup.title)
            jobs_post_list.append(job)

        except Exception as e:
            print(e)

        # finally iterate to the next 
        i = i + 1 

Found 15 results
154: Data Analyst contains 0 mentions of bootcamp
154: Data Analyst contains 2 mentions of SQL
154: Data Analyst contains 3 mentions of Tableau
154: Data Analyst contains 0 mentions of degree
154: Data Analyst contains 19 mentions of data
https://www.indeed.com/viewjob?jk=93e90e27e10d3f1b
Data Analyst contains 0 mentions of bootcamp
Data Analyst contains 2 mentions of SQL
Data Analyst contains 2 mentions of Tableau
Data Analyst contains 0 mentions of degree
Data Analyst contains 18 mentions of data
https://www.indeed.com/viewjob?jk=38d685cb5059bbd5
Data Analyst contains 0 mentions of bootcamp
Data Analyst contains 0 mentions of SQL
Data Analyst contains 0 mentions of Tableau
Data Analyst contains 0 mentions of degree
Data Analyst contains 19 mentions of data
https://www.indeed.com/viewjob?jk=3886612c194ea1de
Data Analyst contains 0 mentions of bootcamp
Data Analyst contains 3 mentions of SQL
Data Analyst contains 0 mentions of Tableau
Data Analyst contains 0 mentions o

## Can iterate through job_post_list and print to console for debugging

In [124]:

print(f"collected {len(jobs_post_list)}")
i1 = 1
for job in jobs_post_list:
    #print(i1)
    #job.desc()
    i1 = i1 + 1

collected 15


## Append the results to a csv if they are unique
Used [this Stack Overflow post](https://stackoverflow.com/questions/34997174/how-to-convert-list-of-model-objects-to-pandas-dataframe) to figure out instantiating data frame from list of objects

In [125]:
print(len(jobs_post_list))
# jobs_df = pd.DataFrame(jobs_post_list, columns=['job_id', 'job_title', 'job_company', 'job_salary',
#                  'location', 'job_date_posted', 'job_company_link', 'job_page_title'])
jobs_df = pd.DataFrame.from_records([job.to_dict() for job in jobs_post_list])
print(jobs_df.head())
jobs_df.to_csv('data/job_posts.csv')

15
             job_id          job_title                    job_company  \
0  93e90e27e10d3f1b  154: Data Analyst                      Dataspace   
1  38d685cb5059bbd5       Data Analyst                           Hulu   
2  3886612c194ea1de       Data Analyst     High Bridge Consulting LLC   
3  ca94f80b933a44ba       Data Analyst  Westinghouse Electric Company   
4  d21e40d081002c62       Data Analyst              Purdue University   

             job_date_posted  \
0  Not specified when posted   
1  Not specified when posted   
2  Not specified when posted   
3  Not specified when posted   
4  Not specified when posted   

                                    job_company_link               job_salary  \
0  http://indeed.com/rc/clk?jk=93e90e27e10d3f1b&f...        $65 - $75 an hour   
1  http://indeed.com/rc/clk?jk=38d685cb5059bbd5&f...  Compensation Not Listed   
2  http://indeed.com/company/High-Bridge-Consulti...        $25 - $35 an hour   
3  http://indeed.com/rc/clk?jk=ca94f80b93